In [ ]:
# Inventory Health
# Query inventory data
query = """
SELECT 
    product_id, product_name,
    units_in_stock, units_on_order,
    reorder_level, discontinued
FROM products;
"""
df = pd.read_sql(query, engine)

# Classify inventory health
def classify_inventory(row):
    if row['discontinued'] == 1:
        return 'Discontinued'
    elif row['reorder_level'] == 0:
        return 'No Reorder Level'
    elif (row['units_in_stock'] + row['units_on_order']) < row['reorder_level']:
        return 'Understock'
    elif row['units_in_stock'] > 2 * row['reorder_level']:
        return 'Overstock'
    else:
        return 'Normal'

df['Inventory Status'] = df.apply(classify_inventory, axis=1)

# Visualize with Plotly
fig = px.bar(
    df.sort_values(by="units_in_stock", ascending=False),
    x='product_name', y='units_in_stock', color='Inventory Status',
    title=' Inventory Health Overview ', labels={'units_in_stock': 'Units in Stock'},
    color_discrete_map={
        'Understock': 'red',
        'Overstock': 'orange',
        'Normal': 'green',
        'No Reorder Level': 'gray',
        'Discontinued': 'black'
    }
)
fig.update_layout(
    xaxis_title=None,
    yaxis_title='Units in Stock',
    xaxis_tickangle=45,
    height=680, width=1200 
)

fig.show()
